In [75]:
import numpy as np
import pandas as pd
from surprise import Dataset
from surprise import Reader
from surprise import SVD, evaluate, accuracy
from surprise import dump
from surprise import SVDpp
from surprise import get_dataset_dir
import io
from collections import defaultdict
from surprise.model_selection import train_test_split

In [100]:
pd.set_option('display.max_rows',300)
pd.set_option('display.max_columns',300)

In [2]:
preds, model = dump.load('final_model')

In [69]:
ls

335k.csv                    final_model
615k.csv                    first_ml_dataset.csv
Data_Gathering.ipynb        good_bad_movie_list.csv
EDA.ipynb                   goodbad.db
Flask Implementation.ipynb  gtag_test.csv
README.md                   item_item_test.csv
Recommender_Modeling.ipynb  ml-latest/
SQL Import.ipynb            ward/
average/


In [70]:
ratings = pd.read_csv('item_item_test.csv')

In [45]:
df = pd.read_csv('good_bad_movie_list.csv')
gb_movies = df[['movieId','title']].copy()

In [54]:
ids = list(gb_movies['movieId'])
titles = list(gb_movies['title'])
zip_list = list(zip(ids,titles))
zip_list

[(56, 'Kids of the Round Table (1995)'),
 (181, 'Mighty Morphin Power Rangers: The Movie (1995)'),
 (810, 'Kazaam (1996)'),
 (1311, 'Santa with Muscles (1996)'),
 (1322, "Amityville 1992: It's About Time (1992)"),
 (1323, 'Amityville 3-D (1983)'),
 (1324, 'Amityville: Dollhouse (1996)'),
 (1325, 'Amityville: A New Generation (1993)'),
 (1328, 'Amityville Curse, The (1990)'),
 (1335, 'Blood Beach (1981)'),
 (1383, 'Adrenalin: Fear the Rush (1996)'),
 (1490, 'B*A*P*S (1997)'),
 (1495, 'Turbo: A Power Rangers Movie (1997)'),
 (1595, 'Free Willy 3: The Rescue (1997)'),
 (1599, 'Steel (1997)'),
 (1679, 'Chairman of the Board (1998)'),
 (1739, '3 Ninjas: High Noon On Mega Mountain (1998)'),
 (1760, 'Spice World (1997)'),
 (1826, "Barney's Great Adventure (1998)"),
 (1853, 'Alan Smithee Film: Burn Hollywood Burn, An (1997)'),
 (1886, 'I Got the Hook Up (1998)'),
 (1981, 'Friday the 13th Part VIII: Jason Takes Manhattan (1989)'),
 (1984, 'Halloween III: Season of the Witch (1982)'),
 (1989, 'P

In [71]:
title_list = []
for i in ratings['movieId']:
    for j in zip_list:
        if i == j[0]:
            title_list.append(j[1])
        else:
            continue

In [62]:
len(title_list)

63163

In [63]:
len(ratings)

63163

In [72]:
ratings['title'] = title_list
ratings.head()

,Unnamed: 0,userId,movieId,rating,timestamp,title
0,90,4,181,0.5,1127946789,Mighty Morphin Power Rangers: The Movie (1995)
1,296,4,1760,0.5,1114050751,Spice World (1997)
2,484,4,3593,1.0,1115782046,Battlefield Earth (2000)
3,1631,19,181,2.0,836433987,Mighty Morphin Power Rangers: The Movie (1995)
4,2655,38,4125,1.0,982853654,Leonard Part 6 (1987)


In [101]:
final = ratings[['userId', 'title', 'rating']]
final.head(125)

,userId,title,rating
0,4,Mighty Morphin Power Rangers: The Movie (1995),0.5
1,4,Spice World (1997),0.5
2,4,Battlefield Earth (2000),1.0
3,19,Mighty Morphin Power Rangers: The Movie (1995),2.0
4,38,Leonard Part 6 (1987),1.0
5,54,Kazaam (1996),3.0
6,56,Freddy Got Fingered (2001),3.5
7,72,Jack and Jill (2011),3.5
8,72,Fifty Shades of Grey (2015),0.5
9,75,Fifty Shades of Grey (2015),2.5


In [176]:
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(final,reader)
trainset = data.build_full_trainset()

In [103]:
# final_model = SVDpp(n_epochs=75, n_factors=50, 
#               lr_all=0.01, reg_all=0.1,random_state=7)


In [104]:
# final_model.fit(trainset)

In [185]:
sim_options = {'name': 'msd', 'user_based': False}
KNN_final_model = KNNBaseline(k=29,sim_options=sim_options)
KNN_final_model.fit(trainset)

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.


In [192]:
mm_raw_id = 'House of the Dead, The (2003)'
mm_inner_id = KNN_final_model.trainset.to_inner_iid(mm_raw_id)
mm_neighbors = KNN_final_model.get_neighbors(mm_inner_id, k=5)
# Convert inner ids of the neighbors into names.
mm_raw = [KNN_final_model.trainset.to_raw_iid(inner_id)
                       for inner_id in mm_neighbors]
# mm_final = (rid_to_name[rid]
#                        for rid in mm_raw)

In [193]:
mm_neighbors

[107, 61, 116, 126, 37]

In [194]:
mm_inner_id

31

In [195]:
mm_raw

['Kids of the Round Table (1995)',
 'Barbie & Her Sisters in the Great Puppy Adventure (2015)',
 'Dance Flick (2009)',
 'Horrors of Spider Island (Ein Toter Hing im Netz) (1960)',
 'Carnosaur 3: Primal Species (1996)']